## 서울시 구별 CCTV 현황 분석하기
- 서울시 각 구별 CCTV 수를 확인하고
- 인구 대비 CCTV 비율을 파악하고 순위 비교
- 인구 대비 CCTV 평균 확인하고 CCTV 부족한 구 확인

### 분석에 사용하는 데이터
- 서울시 자치구 연도별 CCTV 설치 현황
- 서울시 인구 현황

### 데이터 분석 작업 순서
1. 파일로 부터 데이터 읽어 오기

    (1) CCTV 파일 읽어 오기 : 열 이름 변경
    
    (2) 서울시 인구 현황 파일 읽어 오기 : 열 이름 변경
    
    
2. 데이터 파악
    (1) CCTV 데이터 파악
        - CCTV가 가장 많은 구 / 적은 구 파악
        - 최근 CCTV 증가율 구하기
    (2) 서울시 인구 데이터 파악
        - 필요 없는 행 삭제
        - 구 이름 확인
        - NaN 확인하고 포함된 행 삭제
        - 인구 분석 작업
           - 외국인 비율 / 고령자 비율 구하기
           - 인구가 제일 많은 구
           - 외국인이 제일 많은 구
           - 외국인 비율이 제일 높은 구
           - 고령자가 가장 많은 구
3. CCTV 데이터와 인구 데이터 합치기

    (1) 구별 기준으로 데이터 합치기
    
    (2) 의미 없는 열 삭제
    
    (3) 구 별로 인덱스 설정
    
    (4) 상관 관계 분석
    
    
4. 그래프 작성하고 분석

In [30]:
import pandas as pd
import numpy as np

In [3]:
# 1. 파일에서 데이터 읽어 오기
# CCTV 파일 읽어 오기
CCTV_seoul = pd.read_csv('../data/CCTV_in_Seoul.csv', encoding='utf-8')
CCTV_seoul.head()  # 5행만 출력

,기관명,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


In [5]:
CCTV_seoul.tail()  # 마지막 5행

,기관명,소계,2013년도 이전,2014년,2015년,2016년
20,용산구,2096,1368,218,112,398
21,은평구,2108,1138,224,278,468
22,종로구,1619,464,314,211,630
23,중구,1023,413,190,72,348
24,중랑구,916,509,121,177,109


In [6]:
# 열 이름 (컬럼명) 출력
CCTV_seoul.columns

Index(['기관명', '소계', '2013년도 이전', '2014년', '2015년', '2016년'], dtype='object')

In [7]:
CCTV_seoul.columns[0]

'기관명'

In [8]:
# 열 이름 '기관명'을 '구별'로 변경 : rename() 사용
CCTV_seoul.rename(columns={CCTV_seoul.columns[0] : '구별'}, inplace=True)
CCTV_seoul.head()

# inplace=True : 변수의 내용 갱신 (작업 완료) : 없으면 변경되지 않음 (commit)

,구별,소계,2013년도 이전,2014년,2015년,2016년
0,강남구,3238,1292,430,584,932
1,강동구,1010,379,99,155,377
2,강북구,831,369,120,138,204
3,강서구,911,388,258,184,81
4,관악구,2109,846,260,390,613


### folium에 포함된 choropleth() 함수를 사용해서 지도를 표시
- 지도 데이터 파일 (json)
- 시각화하고자 하는 데이터 파일

map.choropleth() 함수(
    geo_data = "지도 데이터 파일 경로",
    
    data = "시각화하고자 하는 데이터",
    
    columns = "지도 데이터와 맵핑할 값, 시각화하고자 하는 변수",
    
    key_on = "feature.데이터 파일과 맵핑할 값",
    
    fill_color = "시각화에 사용될 색상"
    
)

- 인구수에 따른 인구 분포도 - 전국
- 인구 소멸 위기 지역을 나타내는 분포도
- 서울시 구별 살인 분포도
- 서울시 구별 폭력 분포도

In [4]:
# 인구 데이터 파일 읽어오기 (엑셀 파일)
korea_pop = pd.read_excel('../data/korea_population.xls', index_col='ID')
korea_pop.head()

,광역시도,시도,20-39세여자,20-39세합계,65세이상합계,인구수남자,인구수여자,인구수합계,소멸비율,소멸위기지역,y,x,여성비,2030여성비
ID,,,,,,,,,,,,,,
강릉,강원도,강릉시,23098.0,49384.0,37679.0,106231.0,107615,213846.0,1.226041,0,3,11,0.323597,-3.227766
고성(강원),강원도,고성군,2529.0,7023.0,7151.0,15899.0,14215,30114.0,0.707314,1,0,10,-2.796042,-13.989748
동해,강원도,동해시,9753.0,21264.0,15124.0,47166.0,46131,93297.0,1.289738,0,4,11,-0.554680,-4.133747
삼척,강원도,삼척시,7115.0,15823.0,14610.0,35253.0,34346,69599.0,0.973990,1,5,11,-0.651590,-5.033812
속초,강원도,속초시,8752.0,18708.0,12752.0,40288.0,41505,81793.0,1.372647,0,1,10,0.743951,-3.217875


In [11]:
korea_pop.index

Index(['강릉', '고성(강원)', '동해', '삼척', '속초', '양구', '양양', '영월', '원주', '인제',
       ...
       '청주 서원', '영동', '옥천', '음성', '제천', '증평', '진천', '청주 청원', '충주', '청주 흥덕'],
      dtype='object', name='ID', length=252)

In [5]:
import folium
import json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
geo_path = '../data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [31]:
map = folium.Map(location=[36.2002, 127.054], zoom_start=7)

map.choropleth(geo_data = geo_str, 
               data = korea_pop['인구수합계'],
               columns = [korea_pop.index, korea_pop['인구수합계']],
               fill_color = 'YlGnBu',
               key_on = 'feature.id')
map

In [29]:
map = folium.Map(location=[36.2002, 127.054], zoom_start=7)

map.choropleth(geo_data = geo_str, 
               data = korea_pop['소멸위기지역'],
               columns = [korea_pop.index, korea_pop['소멸위기지역']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [15]:
crime = pd.read_csv('../data/crime_in_Seoul.csv', index_col=0)
crime.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율,인구수,CCTV,범죄,검거
구별,,,,,,,,,,,,,,
강남구,1.000000,0.941176,0.916667,1.000000,1.000000,77.728285,85.714286,76.923077,42.857143,86.484594,570500.0,3238,0.007773,85.463066
강동구,0.155620,0.058824,0.166667,0.467528,0.437969,78.846154,100.000000,75.000000,33.347422,82.890855,453233.0,1010,0.002059,85.550226
강북구,0.146974,0.529412,0.416667,0.133118,0.415445,82.352941,92.857143,100.000000,43.096234,88.637222,330192.0,831,0.002627,94.070728
강서구,0.461095,0.470588,0.416667,0.370649,0.614945,72.900763,100.000000,100.000000,60.114504,84.752105,603772.0,911,0.003734,96.572809
관악구,0.628242,0.411765,0.583333,0.589523,0.647479,69.062500,100.000000,88.888889,30.561715,80.109157,525515.0,2109,0.004577,85.212224


In [17]:
crime.index

Index(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구',
       '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구',
       '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype='object', name='구별')

In [16]:
geo_path = '../data/seoul_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [32]:
# 서울시의 중심 위도와 경도로 지정
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

# 컬러맵은 '살인' 발생 건수로 지정
map.choropleth(geo_data = geo_str, 
               data = crime['살인'],
               columns = [crime.index, crime['살인']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [34]:
# 서울시의 중심 위도와 경도로 지정
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

# 컬러맵은 '폭력' 발생 건수로 지정
map.choropleth(geo_data = geo_str, 
               data = crime['폭력'],
               columns = [crime.index, crime['폭력']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map

In [33]:
# 서울시의 중심 위도와 경도로 지정
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)

# 컬러맵은 'CCTV' 발생 건수로 지정
map.choropleth(geo_data = geo_str, 
               data = crime['CCTV'],
               columns = [crime.index, crime['CCTV']],
               fill_color = 'PuRd',
               key_on = 'feature.id')
map